In [1]:
# Set Working Directory
import os
os.chdir('..')

In [2]:
# Load Requirements
import pandas as pd
import numpy as np
import chardet
import zipfile
from functools import reduce
from helper import *

# Obtain data

The data I used for this project came from the [Common Core Dataset](https://nces.ed.gov/ccd/)  managed by the [National Center of Education Statistics](https://nces.ed.gov/). <br/>  
The datasets needed are: <br/>  
- [Universe - District Level - 2009-2010](https://nces.ed.gov/ccd/data/zip/ag092a_txt.zip)
- [Universe - School Level - 2009-2010](https://nces.ed.gov/ccd/data/zip/sc092a_txt.zip)
- [Universe Membership - District Level - 2014-2015](https://nces.ed.gov/ccd/Data/zip/ccd_lea_052_1415_w_0216161a_txt.zip)
- [Universe Directory - District Level - 2014-2015](https://nces.ed.gov/ccd/Data/zip/ccd_lea_029_1415_w_0216161ar_txt.zip)
- [Finance - District Level - 2009-2010](https://nces.ed.gov/ccd/data/zip/SDF101a_txt.zip)
- [Dropout - District Level - 2009-2010](https://nces.ed.gov/ccd/data/zip/dr091a_txt.zip)

I downloaded the data into the 'data' folder, and they were renamed using a common schema ('filetype_fiscalYYYY.txt.zip'). <br/>

Examples:
- 'school_universe_fiscal2010.txt.zip'
- 'directory_fiscal2015.txt.zip'
- 'dropout_fiscal2010.txt.zip'

# Load data into data frames

First, I determined the encoding of the files before loading them into data frames. To accomplish this, I used the [chardet](http://chardet.readthedocs.io/en/latest/usage.html) library.

One key was using a random sample of only 1000 lines to reduce time.

In [3]:
def find_encoding(file, is_zipped=True):
    """
    Return encoding of file

    param string file: Location of file to read (include folder name)
    param bool is_zipped: True if file is zipped, otherwise False
    """
    if is_zipped:
        file_name = file.split('/')[1]
        with zipfile.ZipFile(file, 'r') as archive:
            with archive.open(file_name[:-4]) as raw_data:
                lines = raw_data.readlines()
                sample_lines = np.random.choice(lines, 1000) # use random sample of 1000 lines instead of entire file
                sample_text = b'\n'.join(sample_lines)
                encoding_prediction = chardet.detect(sample_text)
                raw_data.close()
            archive.close()
        encoding_type = encoding_prediction['encoding']
    else:
        with open(file, 'rb') as raw_data:
            lines = raw_data.readlines()
            sample_lines = np.random.choice(lines, 1000) # use random sample of 1000 lines instead of entire file
            sample_text = b'\n'.join(sample_lines)
            encoding_prediction = chardet.detect(sample_text)
            raw_data.close()
        encoding_type = encoding_prediction['encoding']

    return encoding_type

In [4]:
# Test find_encoding function
print('dropout_fiscal2010.txt.zip:', find_encoding(file='data/dropout_fiscal2010.txt.zip', is_zipped=True))
print('directory_fiscal2015.txt.zip:', find_encoding(file='data/directory_fiscal2015.txt.zip', is_zipped=True))

dropout_fiscal2010.txt.zip: ascii
directory_fiscal2015.txt.zip: ascii


I created a function to read the data into dataframes without unzipping the zipped files and using the encoding from earlier.

Since I only used 1000 lines to determine the encoding, it is possible that the encoding is incorrect. To address this, I used a loop with error handling to repeatedly try reading the file with the encoding determined in the find_encoding function until the file is successfully loaded into a dataframe.

In [5]:
def load_data(data_type, year='2010', location='data'):
    """
    Return pandas dataframe of raw data

    param string data_type: Type of data being read
        options: 'finance', 'dropout', 'universe', 'membership'
    param string location: Folder where data is located
    """
    file_in = location + '/' + data_type + '_fiscal' + year + '.txt.zip'
    archive = zipfile.ZipFile(file_in, 'r')
    data_table = None
    while data_table is None: # keep finding encoding until correct encoding is found
        try:
            encoding_type = find_encoding(file_in)
            data_table = pd.read_table(archive.open(data_type + '_fiscal' + year + '.txt'), dtype='str', delimiter='\t', encoding=encoding_type)
        except:
            pass

    return data_table

In [6]:
# Read files into dataframes
finance_data = load_data('finance')
dropout_data = load_data('dropout')
universe_data = load_data('universe')
directory_data = load_data('directory', year='2015')

In [7]:
# Print shapes of data
print('finance_data:', finance_data.shape)
print('dropout_data:', dropout_data.shape)
print('universe_data:', universe_data.shape)
print('directory_data:', directory_data.shape)

finance_data: (18247, 256)
dropout_data: (18439, 10)
universe_data: (18439, 58)
directory_data: (18834, 56)


# Aggregate school level universe data to produce district level data

The demographic breakdown of school districts are not available in the district-level universe data. To obtain district-level demographic data, I aggregated the demographic data from the school level.

In [8]:
def aggregate_school_data(school_columns):
    """
    Return dataframe of numerical columns within school_universe grouped by (sum) LEAID

    param dataframe school_columns: list of columns in the school universe file
    """
    school_universe_data = load_data('school_universe')
    school_universe_data = school_universe_data[school_columns]
    school_universe_data = encode_missing_values(school_universe_data) # description of function in later part of notebook
    for column in school_columns:
        if column != 'LEAID':
            school_universe_data[column] = pd.to_numeric(school_universe_data[column])
    school_universe_data = school_universe_data.groupby('LEAID').sum(skipna=True, min_count=1).reset_index()

    return school_universe_data

In [9]:
# Execute aggregate_school_data function
school_columns = load_columns('school') # reads csv file with list of columns in school_universe file
demographic_data = aggregate_school_data(school_columns)
print('demographic_data:', demographic_data.shape)

demographic_data: (17177, 249)


Because these steps take a long time to complete, I saved the aggregated dataset into a csv file. It is saved in the 'output' folder as 'demographic_data.csv.'

In [10]:
# Save aggregated demographic data into outputs folder for later use
demographic_data.to_csv("output/demographic_data.csv", index=False)

# Merge dataframes

I created a function to merge the dataframes into a single dataframe. The dataframes will be joined by 'LEAID' which is a unique identifier for each school district. Like before, I saved the data in as 'merged_data.csv' in the 'output' folder.

In [11]:
def merge_data(*args):
    """
    Return pandas dataframe of merged dataframe

    param dataframe *args: dataframes of raw data to be joined (typically finance, dropout, and universe)
    """
    data_frames = list(args)
    merged_data = reduce(lambda d1, d2: pd.merge(d1, d2, how='outer', left_on='LEAID', right_on='LEAID'), data_frames)

    return merged_data

In [12]:
# Execute merge_data function
merged_data = merge_data(finance_data, dropout_data, universe_data, demographic_data)
print('merged_data:', merged_data.shape)

merged_data: (18465, 570)


In [13]:
# Save merged_data into outputs folder for later use
merged_data.to_csv("output/merged_data.csv", index=False)

# Remove flag columns

According to documentation, columns that begin with 'FL_' are used to flag data in other columns, and they would not be useful for the analysis. I created a function to remove columns that begin with 'FL_'

In [14]:
def remove_flag_columns(merged_data):
    """
    Return dataframe with flag columns removed

    param dataframe merged_data: merged_data
    """
    flag_columns = [column for column in merged_data.columns.tolist() if column.split('_')[0] == 'FL']
    merged_data.drop(columns=flag_columns, inplace=True)
    return merged_data

In [15]:
merged_data = remove_flag_columns(merged_data)
print('merged_data:', merged_data.shape)

merged_data: (18465, 456)


# Identify NA values

According to documentation, there are multiple types of flagged values.
- -1: missing data
- -2: non-applicable data
- -9, -3, -4: low quality data

I decided to change missing and low quality values to na and change non-applicable data to 0.

In [16]:
def encode_missing_values(raw_data):
    """
    Return dataframe with missing and non applicable values re-encoded as NA and 0

    param DataFrame raw_data: dataframe without re-encoded missing and non applicable values
    """
    reencoded_data = raw_data.copy()
    for column in reencoded_data.columns.tolist():
        missing_values = reencoded_data[column].apply(lambda x: x in ['-1', '-1.0', '-1.00', 'M'] or pd.isnull(x))
        non_applicable_values = reencoded_data[column].apply(lambda x: x in ['-2', '-2.0', '-2.00', 'N'])
        low_quality_values = reencoded_data[column].apply(lambda x: x in ['-9', '-9.0', '-9.00', '-3', '-3.0', '-3.00', '-4', '-4.0', '-4.00'])

        reencoded_data.loc[missing_values, column] = np.nan
        reencoded_data.loc[non_applicable_values, column] = '0'
        reencoded_data.loc[low_quality_values, column] = np.nan

    return reencoded_data

In [17]:
print('missing values before:', pd.isnull(merged_data).sum().sum())
merged_data = encode_missing_values(merged_data)
print('missing values after:', pd.isnull(merged_data).sum().sum())

missing values before: 358416
missing values after: 493470


In [18]:
# Save missing value counts of every column into outputs folder
pd.isnull(merged_data).sum().to_csv('output/missing_values.csv', index=True)

# Create column to represent whether school district is still operational in 5 years

The objective is to predict whether school district will still be operational in 5 years.

I created a column within the data frame to represent whether the school district is still operational in 5 years. To do that, I found school districts in the 2009-2010 universe file that were not found in the 2014-2015 membership file and marked them as being no longer operational. In addition, there are columns representing whether the school district is currently operational ('BOUND09 for 2009-2010 and 'SY_STATUS' for 2014-2015), and I marked school districts that are currently not operational as not being operational in five years as well.

In [19]:
def calc_five_years_operational(merged_data, directory_data):
    """
    Return pandas dataframe of wrangled dataframe

    param dataframe merged_data: dataframe of merged data
    param dataframe directory_data: dataframe with all school districts in 2015
    """
    
    merged_data = merged_data[merged_data['BOUND09'] != '2']
    
    def condition1_generate(x):
        """
        Return boolean of whether school district exists in 5 years

        param string x: LEAID of school district to check
        """
        condition1 = x in directory_data['LEAID'].values
        if condition1:
            condition1 = directory_data.loc[directory_data['LEAID']==x, 'SY_STATUS'].values != '2'
        return condition1

    condition1 = merged_data['LEAID'].apply(condition1_generate)
    condition2 = merged_data['BOUND09'].apply(lambda x: x != '2')
    merged_data['exist_five_years'] = condition1 & condition2
    return merged_data

In [20]:
wrangled_data = calc_five_years_operational(merged_data, directory_data)
print('wrangled_data:', wrangled_data.shape)

wrangled_data: (18167, 457)


/Users/AlbertLee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
# Save wrangled_data into outputs folder
wrangled_data.to_csv('output/wrangled_data.csv', index=False)

# Select Features

There are currently over 450 columns in the dataset. Before proceeding, I selected 30 features to focus on for the analysis based on the following criteria:
- Missing Values: Features should not have too many missing values. I only included columns with less than 1500 missing values.
- Data Leakage: Data should be available at the time of collection and not come from future. I did not choose columns that will lead to data leakage.
- Availability in current dataset: Not all columns in 2009-2010 are still collected by NCES. I only included columns that are still available in the most current common core data.
- Non-Identifier: There are multiple columns that can be used as identifiers (such as 'LEAID' and 'CENSUSID'). These will not be good features to use for the analysis.

The links to documentation for each dataset is located below:
- [Universe - District Level - 2009-2010](https://nces.ed.gov/ccd/pdf/pau092agen.pdf)
- [Universe - School Level - 2009-2010](https://nces.ed.gov/ccd/pdf/INsc90102a.pdf)
- [Universe Membership - District Level - 2014-2015](https://nces.ed.gov/ccd/pdf/2016077_Documentation_062916.pdf)
- [Universe Directory - District Level - 2014-2015](https://nces.ed.gov/ccd/pdf/2016077_Documentation_062916.pdf)
- [Finance - District Level - 2009-2010](https://nces.ed.gov/ccd/pdf/sdf101agen.pdf)
- [Dropout - District Level - 2009-2010](https://nces.ed.gov/ccd/data/zip/2015090_LEA_PU.zip)
- [Universe - District Level - 2014-2015](https://nces.ed.gov/ccd/pdf/2016077_Documentation_062916.pdf)
- [Universe - School Level - 2014-2015](https://nces.ed.gov/ccd/pdf/2016077_Documentation_062916.pdf)
- [Finance - District Level - 2013-2014](https://nces.ed.gov/ccd/data/zip/2016304_F-33_documentation.zip)

In [22]:
# Read list of columns and counts of missing vales
missing_values = pd.read_csv('output/missing_values.csv', names=['column', 'missing_values'])
# remove identifier columns
identifiers = load_columns('identifier')
missing_values = missing_values[missing_values['column'].apply(lambda x: x not in identifiers)]
print(missing_values.shape)

(438, 2)


In [23]:
# Select columns with fewer than 250 missing values
missing_values = missing_values[missing_values['missing_values'] < 250]
print(missing_values.shape)

(40, 2)


In [24]:
print(missing_values)

         column  missing_values
2       FIPST_x             218
7        STNAME             218
8        STABBR             218
9        SCHLEV             218
10       AGCHRT             218
11         YEAR             218
12        CCDNF             218
13      CENFILE             218
14         GSLO             218
15         GSHI             218
17    MEMBERSCH             242
141      WEIGHT             218
142    SURVYEAR              26
143     FIPST_y              26
145      EBS912             241
148       AFGEB             242
151       FIPST              26
152      STID09              26
153      NAME09              26
157    MSTATE09              26
162    LSTATE09              26
165      TYPE09              26
167     CONUM09              28
169       CSA09              28
170      CBSA09              28
171    METMIC09              28
172    ULOCAL09              28
176     BOUND09              26
177      BIEA09              26
178      GSLO09              26
179     

The columns I selected with less than 250 missing values are:
- state name, categorical, STNAME: Public education is managed at the state level, and different states could have different outcomes.
- lowest grade offered in 2009-2010, ordinal, GSLO09: School districts offering different grade levels may have different risks
- highest grade offered in 2009-2010, ordinal, GSHI09: School districts offering different grade levels may have different risks
- total students, numerical (integer), MEMBERSCH: School districts with more students have highest financial needs
- micropolitan or metropolitan area, categorical, METMIC09: School districts in rural areas may have different risks than school districts in urban areas
- whether school district works with Bureau of Indian Education, boolean, BIEA09: School districts operated by Native American Indians may have different outcomes
- none, some, or all schools in school districts are charter schools, categorical, AGCHRT09: School districts working with charter schools may have different outcomes.
- number of schools in school districts, numerical (integer), SCH09: School districts with more schools may have more complex operations
- number of students in english language learning programs, numerical (integer), ELL09: students who are learning english as a new language may have different educational objectives

The following columns also might have been useful, but are not available in the most current data files:
- number of students with disabilities, numerical (integer), SPECED09: students who have disabilities may have different educational objectives
- number of students with standard grade designations, numerical (integer), PK1209: students who don't have standard grade designations may have different educational objectives
- number of students without standard grade designations, numerical (integer), UG09: students who don't have standard grade designations may have different educational objectives
- areas location relative to closest urban cluster, categorical, ULOCAL09: School districts in rural areas may have different risks than school districts in urban areas

In [25]:
# Read list of columns and counts of missing vales
missing_values = pd.read_csv('output/missing_values.csv', names=['column', 'missing_values'])
# remove identifier columns
identifiers = load_columns('identifier')
missing_values = missing_values[missing_values['column'].apply(lambda x: x not in identifiers)]
print(missing_values.shape)

(438, 2)


In [26]:
# Select columns with more than 250 missing values and less than 800 missing values
missing_values = missing_values[(missing_values['missing_values'] >= 250) & (missing_values['missing_values'] < 800)]
print(missing_values.shape)

(15, 2)


In [27]:
print(missing_values)

       column  missing_values
16        V33             398
48        T02             255
49        T06             299
50        T09             299
51        T15             299
52        T40             299
53        T99             299
150    TOTOHC             452
189  ELMTCH09             448
190  SECTCH09             448
191   UGTCH09             250
192  TOTTCH09             391
201  LEAADM09             256
203  SCHADM09             256
205  STUSUP09             536


The columns I selected with more than 250 missing values and less than 800 missing values are:
- local revenue from parent/government contributions, numeric, T02: Amount of local revenue may effect educational outcomes
- local revenue from property taxes, numeric, T06: Amount of local revenue may effect educational outcomes
- local revenue from general sales tax, numeric, T09: Amount of local revenue may effect educational outcomes
- local revenue from public utilities tax, numeric, T15: Amount of local revenue may effect educational outcomes
- local revenue from individual/corporate income tax, numeric, T40: Amount of local revenue may effect educational outcomes
- local revenue from other taxes, numeric, T99: Amount of local revenue may effect educational outcomes
- elementary school teachers, numeric, ELMTCH09: Number of teachers may effect educational outcomes
- secondary school teachers, numeric, SECTCH09: Number of teachers may effect educational outcomes
- teachers in programs without standard grade designations, numeric, UGTCH09: Number of teachers may effect educational outcomes
- total teachers, numeric, TOTTCH09: Number of teachers may effect educational outcomes
- total school district administrators, numeric, LEAADM09: Number of administrators may effect educational outcomes
- total school administrators, numeric, SCHADM09: Number of administrators may effect educational outcomes
- total student support staff, numeric, STUSUP09: Number of support staff may effect educational outcomes

The following columns also might have been useful, but are not available in the most current data files:
- Total Other High School Completion Certificate (OHC) Recipients, numeric (integer), TOTOHC: Students who received certification besides a diploma may have different educational objectives

In [28]:
# Read list of columns and counts of missing vales
missing_values = pd.read_csv('output/missing_values.csv', names=['column', 'missing_values'])
# remove identifier columns
identifiers = load_columns('identifier')
missing_values = missing_values[missing_values['column'].apply(lambda x: x not in identifiers)]
print(missing_values.shape)

(438, 2)


In [29]:
# Select columns with more than 800 missing values and less than 1500 missing values
missing_values = missing_values[(missing_values['missing_values'] >= 800) & (missing_values['missing_values'] < 1500)]
print(missing_values.shape)

(368, 2)


In [30]:
print('column : missing values')
for index, row in missing_values.iterrows():
    print(row['column'], ':', row['missing_values'])

column : missing values
TOTALREV : 855
TFEDREV : 855
C14 : 855
C15 : 855
C16 : 855
C17 : 855
C19 : 855
B11 : 855
C20 : 855
C25 : 855
C36 : 855
B10 : 855
B12 : 855
B13 : 855
TSTREV : 855
C01 : 855
C04 : 855
C05 : 855
C06 : 855
C07 : 855
C08 : 855
C09 : 855
C10 : 855
C11 : 855
C12 : 855
C13 : 855
C35 : 855
C38 : 855
C39 : 855
TLOCREV : 855
D11 : 855
D23 : 855
A07 : 855
A08 : 855
A09 : 855
A11 : 855
A13 : 855
A15 : 855
A20 : 855
A40 : 855
U11 : 855
U22 : 855
U30 : 855
U50 : 855
U97 : 855
C24 : 855
TOTALEXP : 855
TCURELSC : 855
TCURINST : 855
E13 : 855
V91 : 855
V92 : 855
TCURSSVC : 855
E17 : 855
E07 : 855
E08 : 855
E09 : 855
V40 : 855
V45 : 855
V90 : 855
V85 : 855
TCUROTH : 855
E11 : 855
V60 : 855
V65 : 855
TNONELSE : 855
V70 : 855
V75 : 855
V80 : 855
TCAPOUT : 855
F12 : 855
G15 : 855
K09 : 855
K10 : 855
K11 : 855
L12 : 855
M12 : 855
Q11 : 855
I86 : 855
Z32 : 855
Z33 : 855
Z35 : 855
Z36 : 855
Z37 : 855
Z38 : 855
V11 : 855
V13 : 855
V15 : 855
V17 : 855
V21 : 855
V23 : 855
V37 : 855
V29 : 8

The columns I selected with between 800 and 1500 missing values are:
- Total revenue, numeric, TOTALREV: Amount of revenue may effect educational outcomes
- Total federal revenue, numeric, TFEDREV: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE THRU STATE TITLE I, numeric, C14: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE THRU STATE INDIVIDUALS WITH DISABILITIES EDUCATION ACT (IDEA) , numeric, C15: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE THRU STATE MATH SCIENCE AND TEACHER QUALITY, numeric, C16: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE THRU STATE SAFE AND DRUG FREE SCHOOLS, numeric, C17: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE THRU STATE VOCATIONAL AND TECH EDUCATION, numeric, C19: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE DIRECT IMPACT AID, numeric, B10: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE DIRECT INDIAN EDUCATION, numeric, B12: Amount of revenue may effect educational outcomes
- FEDERAL REVENUE DIRECT OTHER, numeric, B13: Amount of revenue may effect educational outcomes
- total state revenue, numeric, TSTREV: Amount of revenue may effect educational outcomes
- STATE REVENUE GENERAL FORMULA ASSISTANCE, numeric, C01: Amount of revenue may effect educational outcomes
- STATE REVENUE STAFF IMPROVEMENT PROGRAMS, numeric, C04: Amount of revenue may effect educational outcomes
- STATE REVENUE SPECIAL EDUCATION PROGRAMS, numeric, C05: Amount of revenue may effect educational outcomes
- STATE REVENUE COMPENSATORY AND BASIC SKILLS PROGRAMS, numeric, C06: Amount of revenue may effect educational outcomes
- STATE REVENUE BILINGUAL EDUCATION PROGRAMS, numeric, C07: Amount of revenue may effect educational outcomes
- STATE REVENUE GIFTED AND TALENTED PROGRAMS, numeric, C08: Amount of revenue may effect educational outcomes
- STATE REVENUE VOCATIONAL EDUCATION PROGRAMS, numeric, C09: Amount of revenue may effect educational outcomes
- STATE REVENUE SCHOOL LUNCH PROGRAMS, numeric, C10: Amount of revenue may effect educational outcomes
- STATE REVENUE CAPITAL OUTLAY AND DEBT SERVICES PROGRAMS, numeric, C11: Amount of revenue may effect educational outcomes
- STATE REVENUE TRANSPORTATION PROGRAMS, numeric, C12: Amount of revenue may effect educational outcomes
- STATE REVENUE OTHER PROGRAMS, numeric, C13: Amount of revenue may effect educational outcomes
- STATE REVENUE NONSPECIFIED, numeric, C35: Amount of revenue may effect educational outcomes
- STATE REVENUE ON BEHALF EMPLOYEE BENEFITS, numeric, C38: Amount of revenue may effect educational outcomes
- STATE REVENUE ON BEHALF NOT EMPLOYEE BENEFITS, numeric, C39: Amount of revenue may effect educational outcomes
- total local revenue, numeric, TLOCREV: Amount of revenue may effect educational outcomes
- LOCAL REVENUE FROM OTHER SCHOOL SYSTEMS, numeric, D11: Amount of revenue may effect educational outcomes
- LOCAL REVENUE FROM CITIES AND COUNTIES, numeric, D23: Amount of revenue may effect educational outcomes
- LOCAL REVENUE TUITION FEES FROM PUPILS AND PARENTS, numeric, A07: Amount of revenue may effect educational outcomes
- LOCAL REVENUE TRANSPORTATION FEES FROM PUPILS AND PARENTS, numeric, A08: Amount of revenue may effect educational outcomes
- LOCAL REVENUE SCHOOL LUNCH, numeric, A09: Amount of revenue may effect educational outcomes
- LOCAL REVENUE TEXTBOOK SALES AND RENTALS, numeric, A11: Amount of revenue may effect educational outcomes
- LOCAL REVENUE DISTRICT ACTIVITY RECEIPTS, numeric, A13: Amount of revenue may effect educational outcomes
- LOCAL REVENUE STUDENTS FEES NONSPECIFIED, numeric, A15: Amount of revenue may effect educational outcomes
- LOCAL REVENUE OTHER SALES AND SERVICES, numeric, A20: Amount of revenue may effect educational outcomes
- LOCAL REVENUE RENTS AND ROYALTIES, numeric, A40: Amount of revenue may effect educational outcomes
- LOCAL REVENUE SALE OF PROPERTY, numeric, U11: Amount of revenue may effect educational outcomes
- LOCAL REVENUE INTEREST EARNINGS, numeric, U22: Amount of revenue may effect educational outcomes
- LOCAL REVENUE FINES AND FORFEITS, numeric, U30: Amount of revenue may effect educational outcomes
- LOCAL REVENUE PRIVATE CONTRIBUTIONS, numeric, U50: Amount of revenue may effect educational outcomes
- LOCAL REVENUE MISCELLANEOUS, numeric, U97: Amount of revenue may effect educational outcomes
- NCES LOCAL REVENUE CENSUS BUREAU STATE REVENUE, numeric, C24: Amount of revenue may effect educational outcomes
- TOTAL EXPENDITURES, numeric, TOTALEXP: Amount of expenditures may effect educational outcomes
- TOTAL CURRENT EXPENDITURES FOR ELEMENTARY/SECONDARY EDUCATION, numeric, TCURELSC: Amount of revenue may effect educational outcomes
- TOTAL CURRENT EXPENDITURES INSTRUCTION, numeric, TCURINST: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES INSTRUCTION, numeric, E13: Amount of expenditures may effect educational outcomes
- PAYMENTS TO PRIVATE SCHOOLS, numeric, V91: Amount of expenditures may effect educational outcomes
- PAYMENTS TO CHARTER SCHOOLS, numeric, V92: Amount of expenditures may effect educational outcomes
- TOTAL CURRENT EXPENDITURES SUPPORT SERVICES, numeric, TCURSSVC: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES PUPILS, numeric, E17: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES INSTRUCTIONAL STAFF, numeric, E07: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES GENERAL ADMINISTRATION, numeric, E08: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES SCHOOL ADMINISTRATION, numeric, E09: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES OPERATION AND MAINTENANCE OF PLANT, numeric, V40: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES STUDENT TRANSPORTATION, numeric, V45: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES BUSINESS/CENTRAL/OTHER, numeric, V90: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES SUPPORT SERVICES NONSPECIFIED, numeric, V85: Amount of expenditures may effect educational outcomes
- TOTAL CURRENT EXPENDITURES OTHER ELEMENTARY/SECONDARY, numeric, TCUROTH: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES FOOD SERVICES, numeric, E11: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES ENTERPRISE OPERATIONS, numeric, V60: Amount of expenditures may effect educational outcomes
- CURRENT EXPENDITURES OTHER ELSEC, numeric, V65: Amount of expenditures may effect educational outcomes
- TOTAL NON-ELEMENTARY/SECONDARY EXPENDITURES, numeric, TNONELSE: Amount of expenditures may effect educational outcomes
- NON ELEMENTARY/SECONDARY EXPENDITURES COMMUNITY SERVICES , numeric, V70: Amount of expenditures may effect educational outcomes
- NON ELEMENTARY/SECONDARY EXPENDITURES ADULT EDUCATION, numeric, V75: Amount of expenditures may effect educational outcomes
- NON ELEMENTARY/SECONDARY EXPENDITURES OTHER, numeric, V80: Amount of expenditures may effect educational outcomes
- TOTAL CAPITAL OUTLAY EXPENDITURES, numeric, TCAPOUT: Amount of expenditures may effect educational outcomes
- CAPITAL OUTLAY CONSTRUCTION, numeric, F12: Amount of expenditures may effect educational outcomes
- CAPITAL OUTLAY LAND AND EXISTING STRUCTURES, numeric, G15: Amount of expenditures may effect educational outcomes
- CAPITAL OUTLAY INSTRUCTIONAL EQUIPMENT, numeric, K09: Amount of expenditures may effect educational outcomes
- CAPITAL OUTLAY OTHER EQUIPMENT, numeric, K10: Amount of expenditures may effect educational outcomes
- CAPITAL OUTLAY NONSPECIFIED EQUIPMENT, numeric, K11: Amount of expenditures may effect educational outcomes
- PAYMENTS TO STATE GOVERNMENTS, numeric, L12: Amount of expenditures may effect educational outcomes
- PAYMENTS TO LOCAL GOVERNMENTS, numeric, M12: Amount of expenditures may effect educational outcomes
- PAYMENTS TO OTHER SCHOOL SYSTEMS, numeric, Q11: Amount of expenditures may effect educational outcomes
- INTEREST ON DEBT, numeric, I86: Amount of expenditures may effect educational outcomes
- TOTAL SALARIES, numeric, Z32: Amount of expenditures may effect educational outcomes
- SALARIES INSTRUCTION, numeric, Z33: Amount of expenditures may effect educational outcomes
- TEACHER SALARIES REGULAR EDUCATION PROGRAMS, numeric, Z35: Amount of expenditures may effect educational outcomes
- TEACHER SALARIES SPECIAL EDUCATION PROGRAMS, numeric, Z36: Amount of expenditures may effect educational outcomes
- TEACHER SALARIES VOCATIONAL EDUCATION PROGRAMS, numeric, Z37: Amount of expenditures may effect educational outcomes
- TEACHER SALARIES OTHER EDUCATION PROGRAMS, numeric, Z38: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES PUPILS, numeric, V11: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES INSTRUCTIONAL STAFF, numeric, V13: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES GENERAL ADMINISTRATION, numeric, V15: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES SCHOOL ADMINISTRATION, numeric, V17: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES OPERATION AND MAINTENANCE OF PLANT, numeric, V21: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES STUDENT TRANSPORTATION, numeric, V23: Amount of expenditures may effect educational outcomes
- SALARIES SUPPORT SERVICES BUSINESS/CENTRAL/OTHER, numeric, V37: Amount of expenditures may effect educational outcomes
- SALARIES FOOD SERVICES, numeric, V29: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS INSTRUCTION, numeric, V10: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES PUPILS, numeric, V12: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES INSTRUCTIONAL STAFF, numeric, V14: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES GENERAL ADMINISTRATION, numeric, V16: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES SCHOOL ADMINISTRATION, numeric, V18: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES OPERATION AND MAINTENANCE OF PLANT, numeric, V22: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES STUDENT TRANSPORTATION, numeric, V24: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS SUPPORT SERVICES BUSINESS/CENTRAL/OTHER, numeric, V38: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS FOOD SERVICES, numeric, V30: Amount of expenditures may effect educational outcomes
- EMPLOYEE BENEFITS ENTERPRISE OPERATIONS, numeric, V32: Amount of expenditures may effect educational outcomes
- TEXTBOOKS, numeric, V93: Amount of expenditures may effect educational outcomes
- LONG TERM DEBT OUTSTANDING AT BEGINNING OF FISCAL YEAR, numeric, _19H: Amount of expenditures may effect educational outcomes
- LONG TERM DEBT ISSUED DURING FISCAL YEAR, numeric, _21F: Amount of expenditures may effect educational outcomes
- LONG TERM DEBT RETIRED DURING FISCAL YEAR, numeric, _31F: Amount of expenditures may effect educational outcomes
- LONG TERM DEBT OUTSTANDING AT END OF FISCAL YEAR, numeric, _41F: Amount of expenditures may effect educational outcomes
- SHORT TERM DEBT OUTSTANDING AT BEGINNING OF FISCAL YEAR, numeric, _61V: Amount of expenditures may effect educational outcomes
- SHORT TERM DEBT OUTSTANDING AT END OF FISCAL YEAR, numeric, _66V: Amount of expenditures may effect educational outcomes
- ASSETS SINKING FUND, numeric, W01: Amount of assets may effect educational outcomes
- ASSETS BOND FUND, numeric, W31: Amount of assets may effect educational outcomes
- ASSETS OTHER FUNDS, numeric, W61: Amount of assets may effect educational outcomes
- AMERICAN RECOVERY AND REINVESTMENT ACT (ARRA) REV TITLE I, numeric, HR1: Amount of revenue may effect educational outcomes
- CURRENT EXP AMERICAN RECOVERY AND REINVESTMENT ACT (ARRA), numeric, HE1: Amount of expenditure may effect educational outcomes
- CAPITAL OUTLAY EXP AMERICAN RECOVERY AND REINVESTMENT ACT (ARRA), numeric, HE2: Amount of expenditure may effect educational outcomes
- Librarians/media specialists, numeric, LIBSPE09: Number of support staff may effect educational outcomes
- American Indian/Alaska Native students, numeric, AM09: School districts with more students have highest financial need
- American Indian/Alaska Native male students, numeric, AMALM09: School districts with more students have highest financial need
- American Indian/Alaska Native female students, numeric, AMALF09: School districts with more students have highest financial need
- Asian/Hawaiian Native/Pacific Islander (5) or Asian (7) students, numeric, ASIAN09: School districts with more students have highest financial need
- Asian/Hawaiian Native/Pacific Islander (5) or Asian (7) male students, numeric, ASALM09: School districts with more students have highest financial need
- Asian/Hawaiian Native/Pacific Islander (5) or Asian (7)  female students, numeric, ASALF09: School districts with more students have highest financial need
- Hispanic students, numeric, HISP09: School districts with more students have highest financial need
- Hispanic male students, numeric, HIALM09: School districts with more students have highest financial need
- Hispanic female students, numeric, HIALF09: School districts with more students have highest financial need
- Black non Hispanic students, numeric, BLACK09: School districts with more students have highest financial need
- Black non Hispanic male students, numeric, BLALM09: School districts with more students have highest financial need
- Black non Hispanic female students, numeric, BLALF09: School districts with more students have highest financial need
- White students, numeric, WHITE09: School districts with more students have highest financial need
- White male students, numeric, WHALM09: School districts with more students have highest financial need
- White female students, numeric, WHALF09: School districts with more students have highest financial need
- Hawaiian Native/Pacific Islander students, numeric, PACIFIC09: School districts with more students have highest financial need
- Hawaiian Native/Pacific Islander male students, numeric, HPALM09: School districts with more students have highest financial need
- Hawaiian Native/Pacific Islander female students, numeric, HPALF09: School districts with more students have highest financial need
- Two or more races students, numeric, TR09: School districts with more students have highest financial need
- Two or more races male students, numeric, TRALM09: School districts with more students have highest financial need
- Two or more races female students, numeric, TRALF09: School districts with more students have highest financial need


The following columns also might have been useful, but are not available in the most current data files:
- Total Diploma Count, numeric (integer), TOTDPL: Schools giving more diplomas may be at less risk
- Total Averaged Freshmen Graduation Rate (AFGR), numeric (integer), AFGR: High graduation rates may show school district is at lower risk

# Rename columns

I renamed the columns using more interpretable names.

In [31]:
# create dictionaries mapping old column names to new column names
few_missing_values_columns = {'STNAME': 'state_name', 
                              'GSLO09': 'lowest_grade', 
                              'GSHI09': 'highest_grade', 
                              'MEMBERSCH': 'total_students', 
                              'METMIC09': 'metro_micro', 
                              'BIEA09': 'bureau_indian_education', 
                              'AGCHRT09': 'charter_status', 
                              'SCH09': 'total_schools', 
                              'ELL09': 'english_language_learners'}

normal_missing_value_columns = {'T02': 'local_revenue_parent_government_contributions', 
                                'T06': 'local_revenue_property_tax', 
                                'T09': 'local_revenue_sales_tax', 
                                'T15': 'local_revenue_utilities_tax', 
                                'T40': 'local_revenue_income_tax', 
                                'T99': 'local_revenue_other_tax', 
                                'ELMTCH09': 'teachers_elementary', 
                                'SECTCH09': 'teachers_secondary', 
                                'UGTCH09': 'teachers_ungraded', 
                                'TOTTCH09': 'teachers_total', 
                                'LEAADM09': 'administrators_district', 
                                'SCHADM09': 'administrators_school', 
                                'STUSUP09': 'staff_student_support'}

many_missing_value_columns = {'TOTALREV': 'total_revenue', 
                              'TFEDREV': 'total_federal_revenue', 
                              'C14': 'federal_revenue_state_title_i', 
                              'C15': 'federal_revenue_ideas', 
                              'C16': 'federal_revenue_math_science_quality', 
                              'C17': 'federal_revenue_drug_free', 
                              'C19': 'federal_revenue_vocational_tech_training', 
                              'B10': 'federal_revenue_impact_aid', 
                              'B12': 'federal_revenue_indian_education', 
                              'B13': 'federal_revenue_other', 
                              'TSTREV': 'total_state_revenue', 
                              'C01': 'state_revenue_general_formula_assistance', 
                              'C04': 'state_revenue_staff_improvement', 
                              'C05': 'state_revenue_special_education',
                              'C06': 'state_revenue_compensatory_basic_training', 
                              'C07': 'state_revenue_bilingual_education', 
                              'C08': 'state_revenue_gifted_talented', 
                              'C09': 'state_revenue_vocational_education', 
                              'C10': 'state_revenue_school_lunch', 
                              'C11': 'state_revenue_capital_outlay_debt_services', 
                              'C12': 'state_revenue_transportation', 
                              'C13': 'state_revenue_other', 
                              'C35': 'state_revenue_nonspecified', 
                              'C38': 'state_revenue_employee_benefits', 
                              'C39': 'state_revenue_not_employee_benefits', 
                              'TLOCREV': 'total_local_revenue', 
                              'D11': 'local_revenue_other_school_systems', 
                              'D23': 'local_revenue_cities_counties',
                              'A07': 'local_revenue_tuition_fee_pupils_parents', 
                              'A08': 'local_revenue_transportation_fee_pupil_parents', 
                              'A09': 'local_revenue_school_lunch', 
                              'A11': 'local_revenue_textbook_sale_rental', 
                              'A13': 'local_revenue_district_activity_receipt', 
                              'A15': 'local_revenue_student_fee_nonspecified', 
                              'A20': 'local_revenue_other_sales_services', 
                              'A40': 'local_revenue_rent_royalties', 
                              'U11': 'local_revenue_property_sales', 
                              'U22': 'local_revenue_interest_earnings', 
                              'U30': 'local_revenue_fines_forfeits', 
                              'U50': 'local_revenue_private_contributions', 
                              'U97': 'local_revenue_miscellaneous', 
                              'C24': 'local_revenue_NCES', 
                              'TOTALEXP': 'total_expenditure',
                              'TCURELSC': 'total_expenditure_elementary_secondary', 
                              'TCURINST': 'total_expenditure_instruction', 
                              'E13': 'total_expenditure_instruction_public', 
                              'V91': 'expenditure_private_school', 
                              'V92': 'expenditure_charter_school', 
                              'TCURSSVC': 'total_expenditure_support_services', 
                              'E17': 'expenditure_support_services_pupils', 
                              'E07': 'expenditure_support_services_instructional_staff', 
                              'E08': 'expenditure_support_services_general_administration', 
                              'E09': 'expenditure_support_services_school_administration', 
                              'V40': 'expenditure_support_services_maintenance', 
                              'V45': 'expenditure_support_services_transportation', 
                              'V90': 'expenditure_support_services_business',
                              'V85': 'expenditure_support_services_nonspecified', 
                              'TCUROTH': 'total_expenditure_other_elementary_secondary', 
                              'E11': 'expenditure_food_service', 
                              'V60': 'expenditure_enterprise', 
                              'V65': 'expenditure_other_elementary_secondary', 
                              'TNONELSE': 'total_expenditure_non_elementary_secondary', 
                              'V70': 'expenditure_non_elementary_secondary_community_service', 
                              'V75': 'expenditure_non_elementary_secondary_adult_education', 
                              'V80': 'expenditure_non_elementary_secondary_other', 
                              'TCAPOUT': 'total_expenditure_capital_outlay', 
                              'F12': 'expenditure_capital_outlay_construction', 
                              'G15': 'expenditure_capital_outlay_land_existing_structures',
                              'K09': 'expenditure_capital_outlay_instruction_equipment', 
                              'K10': 'expenditure_capital_outlay_other_equipment', 
                              'K11': 'expenditure_capital_outlay_nonspecified', 
                              'L12': 'payments_state_government', 
                              'M12': 'payments_local_government', 
                              'Q11': 'payments_other_school_systems', 
                              'I86': 'interest_on_debt', 
                              'Z32': 'total_salaries', 
                              'Z33': 'salaries_instruction', 
                              'Z35': 'salaries_regular_education', 
                              'Z36': 'salaries_special_education', 
                              'Z37': 'salaries_vocationall_education', 
                              'Z38': 'salaries_other_education', 
                              'V11': 'salaries_support_services_pupils', 
                              'V13': 'salaries_support_services_instructional_staff', 
                              'V15': 'salaries_support_services_general_administration', 
                              'V17': 'salaries_support_services_school_administration',
                              'V21': 'salaries_support_services_maintenance', 
                              'V23': 'salaries_support_transportation', 
                              'V37': 'salaries_support_services_business',
                              'V29': 'salaries_food_service',
                              'V10': 'employee_benefits_instruction', 
                              'V12': 'employee_benefits_support_services_pupil', 
                              'V14': 'employee_benefits_support_services_instructional_staff', 
                              'V16': 'employee_benefits_support_services_general_administration', 
                              'V18': 'employee_benefits_support_services_school_administration', 
                              'V22': 'employee_benefits_support_services_maintenance', 
                              'V24': 'employee_benefits_support_transportation', 
                              'V38': 'employee_benefits_support_services_business', 
                              'V30': 'employee_benefits_food_service', 
                              'V32': 'employee_benefits_enterprise', 
                              'V93': 'textbooks', 
                              '_19H': 'long_term_debt_outstanding_beginning_fiscal_year',
                              '_21F': 'long_term_debt_issued_during_fiscal_year', 
                              '_31F': 'long_term_debt_retired_during_fiscal_year', 
                              '_41F': 'long_term_debt_outstanding_end_fiscal_year', 
                              '_61V': 'short_term_debt_outstanding_beginning_fiscal_year', 
                              '_66V': 'short_term_debt_outstanding_end_fiscal_year', 
                              'W01': 'assets_sinking_fund', 
                              'W31': 'assets_bond_fund', 
                              'W61': 'assets_other_funds', 
                              'HR1': 'ARRA_revenue', 
                              'HE1': 'ARRA_current_expenditures', 
                              'HE2': 'ARRA_capital_outlay_expenditures', 
                              'LIBSPE09': 'librarian_media_specialists', 
                              'AM09': 'american_indian_alaskan_native_students', 
                              'AMALM09': 'american_indian_alaskan_native_male_students', 
                              'AMALF09': 'american_indian_alaskan_native_female_students', 
                              'ASIAN09': 'asian_hawaiian_native_pacific_islander_students', 
                              'ASALM09': 'asian_hawaiian_native_pacific_islander_male_students', 
                              'ASALF09': 'asian_hawaiian_native_pacific_islander_female_students', 
                              'HISP09': 'hispanic_students', 
                              'HIALM09': 'hispanic_male_students', 
                              'HIALF09': 'hispanic_female_students', 
                              'BLACK09': 'black_non_hispanic_students', 
                              'BLALM09': 'black_non_hispanic_male_students', 
                              'BLALF09': 'black_non_hispanic_female_students', 
                              'WHITE09': 'white_students', 
                              'WHALM09': 'white_male_students', 
                              'WHALF09': 'white_female_students', 
                              'PACIFIC09': 'hawaiian_native_pacific_islander_students', 
                              'HPALM09': 'hawaiian_native_pacific_islander_male_students', 
                              'HPALF09': 'hawaiian_native_pacific_islander_female_students', 
                              'TR09': 'mixed_race_students', 
                              'TRALM09': 'mixed_race_male_students',
                              'TRALF09': 'mixed_race_female_students'}

In [32]:
# create list of columns to keep
columns_to_keep = ['LEAID', 'NAME', 'exist_five_years']
for column in few_missing_values_columns.keys():
    columns_to_keep.append(column)
for column in normal_missing_value_columns.keys():
    columns_to_keep.append(column)
for column in many_missing_value_columns.keys():
    columns_to_keep.append(column)
print(len(columns_to_keep))

159


In [33]:
# remove unnecessary columns
wrangled_data_ii = wrangled_data[columns_to_keep].copy()

# rename columns
wrangled_data_ii.rename(columns=few_missing_values_columns, inplace=True)
wrangled_data_ii.rename(columns=normal_missing_value_columns, inplace=True)
wrangled_data_ii.rename(columns=many_missing_value_columns, inplace=True)

wrangled_data_ii['state_name'] = wrangled_data_ii['state_name'].str.lower()

print(wrangled_data_ii.sample(5))

         LEAID                            NAME  exist_five_years  state_name  \
17401  5301080  CENTRAL KITSAP SCHOOL DISTRICT              True  washington   
16720  4900069   UTAH SCHOOLS FOR DEAF & BLIND              True        utah   
15436  4800068   TREETOPS SCHOOL INTERNATIONAL              True       texas   
12906  3904708           PIKE-DELTA-YORK LOCAL              True        ohio   
12602  3904362                     BEXLEY CITY              True        ohio   

      lowest_grade highest_grade total_students metro_micro  \
17401           PK            12          11365           1   
16720           PK            12            428           1   
15436           KG            12            367           1   
12906           PK            12           1318           1   
12602           PK            12           2080           1   

      bureau_indian_education charter_status            ...              \
17401                       2              3            ...      

In [34]:
# save dataframe to csv
wrangled_data_ii.to_csv('output/wrangled_data_ii.csv', index=False)